In [2]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [4]:
mpg.hwy

Column<b'hwy'>

In [5]:
mpg.select(mpg.cty, mpg.hwy, mpg.model)

DataFrame[cty: bigint, hwy: bigint, model: string]

In [6]:
mpg.select(mpg.cty, mpg.hwy, mpg.model).show(10)

+---+---+----------+
|cty|hwy|     model|
+---+---+----------+
| 18| 29|        a4|
| 21| 29|        a4|
| 20| 31|        a4|
| 21| 30|        a4|
| 16| 26|        a4|
| 18| 26|        a4|
| 18| 27|        a4|
| 18| 26|a4 quattro|
| 16| 25|a4 quattro|
| 20| 28|a4 quattro|
+---+---+----------+
only showing top 10 rows

